In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [9]:
%%time
import pandas as pd
import numpy as np
import time
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

df = pd.read_csv("tmbExport.csv")
print('CSV retrieved.')
df1 = df.sort_values(['siteid'])
print('Values sorted by siteid')
df1.reset_index(inplace = True, drop = True)
df1['sendnotices.1'] = df1['sendnotices.1'].fillna(0).astype(bool)
df1['sendnotices'] = df1['sendnotices'].fillna(0).astype(bool)
print('Filled null sendnotices row values with zeros')
print('Changed sendnotice rows to booleans')
targetColumns = ["utility", "billing", "notice", "envelope", "sId", "sCompany", "sAddress", "sCity", "sState", "sZip", "mCompany", "mContact", "mAddress", "mCity", "mState", "mZip", "hId1", "due1", "size1", "model1", "serial1", "cat1", "loc1", "testCompany1", "testCompanyPhone1", "hId2", "due2", "size2", "model2", "serial2", "cat2", "loc2", "testCompany2", "testCompanyPhone2", "hId3", "due3", "size3", "model3", "serial3", "cat3", "loc3", "testCompany3", "testCompanyPhone3", "hId4", "due4", "size4", "model4", "serial4", "cat4", "loc4", "testCompany4", "testCompanyPhone4", "hId5", "due5", "size5", "model5", "serial5", "cat5", "loc5", "testCompany5", "testCompanyPhone5", "hId6", "due6", "size6", "model6", "serial6", "cat6", "loc6", "testCompany6", "testCompanyPhone6", "hId7", "due7", "size7", "model7", "serial7", "cat7", "loc7", "testCompany7", "testCompanyPhone7", "hId8", "due8", "size8", "model8", "serial8", "cat8", "loc8", "testCompany8", "testCompanyPhone8", "hId9", "due9", "size9", "model9", "serial9", "cat9", "loc9", "testCompany9", "testCompanyPhone9", "hId10", "due10", "size10", "model10", "serial10", "cat10", "loc10", "testCompany10", "testCompanyPhone10", "hId11", "due11", "size11", "model11", "serial11", "cat11", "loc11", "testCompany11", "testCompanyPhone11", "hId12", "due12", "size12", "model12", "serial12", "cat12", "loc12", "testCompany12", "testCompanyPhone12", "hId13", "due13", "size13", "model13", "serial13", "cat13", "loc13", "testCompany13", "testCompanyPhone13", "hId14", "due14", "size14", "model14", "serial14", "cat14", "loc14", "testCompany14", "testCompanyPhone14", "hId15", "due15", "size15", "model15", "serial15", "cat15", "loc15", "testCompany15", "testCompanyPhone15", "hId16", "due16", "size16", "model16", "serial16", "cat16", "loc16", "testCompany16", "testCompanyPhone16", "hId17", "due17", "size17", "model17", "serial17", "cat17", "loc17", "testCompany17", "testCompanyPhone17", "hId18", "due18", "size18", "model18", "serial18", "cat18", "loc18", "testCompany18", "testCompanyPhone18", "hId19", "due19", "size19", "model19", "serial19", "cat19", "loc19", "testCompany19", "testCompanyPhone19", "hId20", "due20", "size20", "model20", "serial20", "cat20", "loc20", "testCompany20", "testCompanyPhone20", "hId21", "due21", "size21", "model21", "serial21", "cat21", "loc21", "testCompany21", "testCompanyPhone21", "hId22", "due22", "size22", "model22", "serial22", "cat22", "loc22", "testCompany22", "testCompanyPhone22", "hId23", "due23", "size23", "model23", "serial23", "cat23", "loc23", "testCompany23", "testCompanyPhone23", "hId24", "due24", "size24", "model24", "serial24", "cat24", "loc24", "testCompany24", "testCompanyPhone24", "hId25", "due25", "size25", "model25", "serial25", "cat25", "loc25", "testCompany25", "testCompanyPhone25", "hId26", "due26", "size26", "model26", "serial26", "cat26", "loc26", "testCompany26", "testCompanyPhone26", "hId27", "due27", "size27", "model27", "serial27", "cat27", "loc27", "testCompany27", "testCompanyPhone27", "hId28", "due28", "size28", "model28", "serial28", "cat28", "loc28", "testCompany28", "testCompanyPhone28", "hId29", "due29", "size29", "model29", "serial29", "cat29", "loc29", "testCompany29", "testCompanyPhone29", "hId30", "due30", "size30", "model30", "serial30", "cat30", "loc30", "testCompany30", "testCompanyPhone30", "hId31", "due31", "size31", "model31", "serial31", "cat31", "loc31", "testCompany31", "testCompanyPhone31", "hId32", "due32", "size32", "model32", "serial32", "cat32", "loc32", "testCompany32", "testCompanyPhone32", "hId33", "due33", "size33", "model33", "serial33", "cat33", "loc33", "testCompany33", "testCompanyPhone33", "hId34", "due34", "size34", "model34", "serial34", "cat34", "loc34", "testCompany34", "testCompanyPhone34", "hId35", "due35", "size35", "model35", "serial35", "cat35", "loc35", "testCompany35", "testCompanyPhone35", "hId36", "due36", "size36", "model36", "serial36", "cat36", "loc36", "testCompany36", "testCompanyPhone36", "hId37", "due37", "size37", "model37", "serial37", "cat37", "loc37", "testCompany37", "testCompanyPhone37", "hId38", "due38", "size38", "model38", "serial38", "cat38", "loc38", "testCompany38", "testCompanyPhone38", "hId39", "due39", "size39", "model39", "serial39", "cat39", "loc39", "testCompany39", "testCompanyPhone39", "hId40", "due40", "size40", "model40", "serial40", "cat40", "loc40", "testCompany40", "testCompanyPhone40", "hId41", "due41", "size41", "model41", "serial41", "cat41", "loc41", "testCompany41", "testCompanyPhone41", "hId42", "due42", "size42", "model42", "serial42", "cat42", "loc42", "testCompany42", "testCompanyPhone42"]
dfTarget = pd.DataFrame(columns = targetColumns)
dfFinal = pd.DataFrame(columns = targetColumns)
print('Structured intermediary target dataframe')
print('Structured final dataframe')
utilityEnv = ['Archon', 'Barrington', 'Carson City', 'Cedar Falls', 'Elk Grove', 'Johnstown', 'Kewanee', 'Lawrence', 'Marion Utilities', 'Mill Creek', 'New Braunfels', 'Owensboro', 'Prosser', 'Richton Park', 'Signal Hill']
print('Got list of cities that supply their own envelopes')
userCheck = []
userCheckSIDs = []

df1 = df1[~df1.municipality.str.contains("Sample")]
df1 = df1[~df1.municipality.str.contains("Demo")]
df1 = df1[~df1.municipality.str.contains("NOT UTILITY")]
df1 = df1[~df1.municipality.str.contains("Available")]
df1 = df1[~df1['address.1'].str.contains("@", na=False)]
df1.reset_index(drop=True, inplace=True)

dfSendSite = df1[df1['sendnotices']]
for i in dfSendSite.index :
    if (dfSendSite.at[i, 'siteid'] in dfFinal['sId'].values) :
        continue
    else :
        dfTarget.loc[0, 'utility'] = dfSendSite.loc[i, 'municipality']
        dfTarget.loc[0, 'sId'] = dfSendSite.loc[i, 'siteid']
        dfTarget.loc[0, ['sCompany', 'mCompany']] = dfSendSite.loc[i, 'company']
        dfTarget.loc[0, 'mContact'] = dfSendSite.loc[i, 'contact']
        dfTarget.loc[0, ['sAddress', 'mAddress']] = dfSendSite.loc[i, 'address']
        dfTarget.loc[0, ['sCity', 'mCity']] = dfSendSite.loc[i, 'city']
        dfTarget.loc[0, ['sState', 'mState']] = dfSendSite.loc[i, 'state']
        dfTarget.loc[0, ['sZip', 'mZip']] = dfSendSite.loc[i, 'zip']
        dfTemp = dfSendSite[dfSendSite['siteid'] == dfSendSite.at[i, 'siteid']]
        dfTempTrimmed = dfTemp.drop_duplicates('hazid')
        for i in dfTempTrimmed.index :
            ci = 16
            while ci < 100000 :
                if (np.isnan(dfTarget.at[0, dfTarget.columns[ci]])) :
                    dfTarget.at[0, dfTarget.columns[ci]] = dfTempTrimmed.at[i, 'hazid']
                    dfTarget.at[0, dfTarget.columns[ci + 1]] = dfTempTrimmed.at[i, 'testdue']
                    dfTarget.at[0, dfTarget.columns[ci + 2]] = dfTempTrimmed.at[i, 'devsize']
                    dfTarget.at[0, dfTarget.columns[ci + 3]] = dfTempTrimmed.at[i, 'model']
                    dfTarget.at[0, dfTarget.columns[ci + 4]] = dfTempTrimmed.at[i, 'serialnum']
                    dfTarget.at[0, dfTarget.columns[ci + 5]] = dfTempTrimmed.at[i, 'hazardcat']
                    dfTarget.at[0, dfTarget.columns[ci + 6]] = dfTempTrimmed.at[i, 'location']
                    dfTarget.at[0, dfTarget.columns[ci + 7]] = dfTempTrimmed.at[i, 'lasttestcompany']
                    dfTarget.at[0, dfTarget.columns[ci + 8]] = dfTempTrimmed.at[i, 'lasttestcompanyphone']
                    ci += 1000000
                else :
                    ci += 9
        dfTarget.at[0, 'utility'] = dfSendSite.at[i, 'municipality']
        dfFinal = dfFinal.append(dfTarget, ignore_index = True)
        print('Added letter going to site: ' + str(dfTarget.at[0, 'sId']))
        dfTarget.drop(dfTarget.index[0], inplace = True)
for i in df1.index :
    if df1.at[i, 'sendnotices.1'] :
        mask1 = df1['siteid'] == df1.at[i, 'siteid']
        mask2 = df1['sendnotices.1']
        dfTemp = df1[mask1 & mask2]
        for j in dfTemp.index :
            if (dfTemp.at[j, 'address.1'] in dfFinal[dfFinal['sId'] == dfTemp.at[j, 'siteid']]['mAddress'].values) :
                continue
            else :
                dfTarget.at[0, 'utility'] = dfTemp.at[j, 'municipality']
                dfTarget.at[0, 'sId'] = dfTemp.at[j, 'siteid']
                dfTarget.at[0, 'sCompany'] = dfTemp.at[j, 'company']
                dfTarget.at[0, 'sAddress'] = dfTemp.at[j, 'address']
                dfTarget.at[0, 'sCity'] = dfTemp.at[j, 'city']
                dfTarget.at[0, 'sState'] = dfTemp.at[j, 'state']
                dfTarget.at[0, 'sZip'] = dfTemp.at[j, 'zip']
                dfTarget.at[0, 'mCompany'] = dfTemp.at[j, 'company.1']
                dfTarget.at[0, 'mContact'] = dfTemp.at[j, 'contact.1']
                dfTarget.at[0, 'mAddress'] = dfTemp.at[j, 'address.1']
                dfTarget.at[0, 'mCity'] = dfTemp.at[j, 'city.1']
                dfTarget.at[0, 'mState'] = dfTemp.at[j, 'state.1']
                dfTarget.at[0, 'mZip'] = dfTemp.at[j, 'zip.1']
                maskAddr = dfTemp['address.1'] == dfTemp.at[j, 'address.1']
                dfTempTrimmed = dfTemp[maskAddr]
                for i in dfTempTrimmed.index :
                    ci = 16
                    while ci < 100000 :
                        if (np.isnan(dfTarget.at[0, dfTarget.columns[ci]])) :
                            dfTarget.at[0, dfTarget.columns[ci]] = dfTempTrimmed.at[i, 'hazid']
                            dfTarget.at[0, dfTarget.columns[ci + 1]] = dfTempTrimmed.at[i, 'testdue']
                            dfTarget.at[0, dfTarget.columns[ci + 2]] = dfTempTrimmed.at[i, 'devsize']
                            dfTarget.at[0, dfTarget.columns[ci + 3]] = dfTempTrimmed.at[i, 'model']
                            dfTarget.at[0, dfTarget.columns[ci + 4]] = dfTempTrimmed.at[i, 'serialnum']
                            dfTarget.at[0, dfTarget.columns[ci + 5]] = dfTempTrimmed.at[i, 'hazardcat']
                            dfTarget.at[0, dfTarget.columns[ci + 6]] = dfTempTrimmed.at[i, 'location']
                            dfTarget.at[0, dfTarget.columns[ci + 7]] = dfTempTrimmed.at[i, 'lasttestcompany']
                            dfTarget.at[0, dfTarget.columns[ci + 8]] = dfTempTrimmed.at[i, 'lasttestcompanyphone']
                            ci += 1000000
                        else :
                            ci += 9
                dfFinal = dfFinal.append(dfTarget, ignore_index = True)
                print('Added letter going to mailing address: ' + str(dfTarget.at[0, 'mAddress']))
                dfTarget.drop(dfTarget.index[0], inplace = True)
mask1 = df1['sendnotices'] == False
mask2 = df1['sendnotices.1'] == False
dfBothZero = df1[mask1 & mask2]
for i in dfBothZero.index :
    if (dfBothZero.at[i, 'siteid'] in dfFinal['sId'].values) :
        continue
    else :
        dfTarget.loc[0, 'utility'] = dfBothZero.loc[i, 'municipality']
        dfTarget.loc[0, 'sId'] = dfBothZero.loc[i, 'siteid']
        dfTarget.loc[0, ['sCompany', 'mCompany']] = dfBothZero.loc[i, 'company']
        dfTarget.loc[0, 'mContact'] = dfBothZero.loc[i, 'contact']
        dfTarget.loc[0, ['sAddress', 'mAddress']] = dfBothZero.loc[i, 'address']
        dfTarget.loc[0, ['sCity', 'mCity']] = dfBothZero.loc[i, 'city']
        dfTarget.loc[0, ['sState', 'mState']] = dfBothZero.loc[i, 'state']
        dfTarget.loc[0, ['sZip', 'mZip']] = dfBothZero.loc[i, 'zip']
        dfTemp = dfBothZero[dfBothZero['siteid'] == dfBothZero.at[i, 'siteid']]
        dfTempTrimmed = dfTemp.drop_duplicates('hazid')
        for i in dfTempTrimmed.index :
            ci = 16
            while ci < 100000 :
                if (np.isnan(dfTarget.at[0, dfTarget.columns[ci]])) :
                    dfTarget.at[0, dfTarget.columns[ci]] = dfTempTrimmed.at[i, 'hazid']
                    dfTarget.at[0, dfTarget.columns[ci + 1]] = dfTempTrimmed.at[i, 'testdue']
                    dfTarget.at[0, dfTarget.columns[ci + 2]] = dfTempTrimmed.at[i, 'devsize']
                    dfTarget.at[0, dfTarget.columns[ci + 3]] = dfTempTrimmed.at[i, 'model']
                    dfTarget.at[0, dfTarget.columns[ci + 4]] = dfTempTrimmed.at[i, 'serialnum']
                    dfTarget.at[0, dfTarget.columns[ci + 5]] = dfTempTrimmed.at[i, 'hazardcat']
                    dfTarget.at[0, dfTarget.columns[ci + 6]] = dfTempTrimmed.at[i, 'location']
                    dfTarget.at[0, dfTarget.columns[ci + 7]] = dfTempTrimmed.at[i, 'lasttestcompany']
                    dfTarget.at[0, dfTarget.columns[ci + 8]] = dfTempTrimmed.at[i, 'lasttestcompanyphone']
                    ci += 1000000
                else :
                    ci += 9
        dfTarget.at[0, 'utility'] = dfBothZero.at[i, 'municipality']
        dfFinal = dfFinal.append(dfTarget, ignore_index = True)
        print('\'sendnotice\' field was 0 for both site and hazard. Added letter going to site: ' + str(dfTarget.at[0, 'sId']))
        dfTarget.drop(dfTarget.index[0], inplace = True)
for i in dfFinal.index :
    if dfFinal.at[i, 'utility'].startswith('Archon') :
        dfFinal.at[i, 'billing'] = 'Archon'
    else :
        dfFinal.at[i, 'billing'] = 'ABF'
dfFinal['envelope'] = 'ABF'
for i in dfFinal[dfFinal['utility'].str.contains('|'.join(utilityEnv))].index :
    print('Setting envelope for ' + str(dfFinal.at[i, 'utility']))
    dfFinal.at[i, 'envelope'] = dfFinal.at[i, 'utility']
print('Set billing to ABF or Archon')
print('Deleting non-real utilities')
dfFinal.sort_values(['sId'], inplace=True)
dfFinal.reset_index(drop=True, inplace=True)
print('Saving file...')
dfFinal.to_excel('output.xlsx')
dfOnlyDups = dfFinal[dfFinal.duplicated(subset = 'sId', keep = False)]
for i in dfOnlyDups.index :
    maskSID = dfOnlyDups['sId'] == dfOnlyDups.at[i, 'sId']
    dfTrimmed = dfOnlyDups[maskSID]
    for j in dfTrimmed.index :
        for x in dfTrimmed.index :
            if (j == x) :
                continue
            if (fuzz.ratio(dfTrimmed.at[j, 'mAddress'].lower(), dfTrimmed.at[x, 'mAddress'].lower()) > 60) :
                if (fuzz.ratio(dfTrimmed.at[j, 'mCity'].lower(), dfTrimmed.at[x, 'mCity'].lower()) < 90) :
                    continue
                elif not (dfTrimmed.at[j, 'sId'] in userCheckSIDs) :
                    userCheckSIDs.append(dfTrimmed.at[j, 'sId'])
                    userCheck.append({'sId':dfTrimmed.at[j, 'sId'], 'm1':dfTrimmed.at[j, 'mAddress'], 'm2':dfTrimmed.at[x, 'mAddress'], 'mC1':dfTrimmed.at[j, 'mCompany'], 'mC2':dfTrimmed.at[x, 'mCompany']})
                x += 1
            x += 1
dash = '-' * 100
print('\n\n\n\n\n\n' + dash)
print('These addresses looked similar')
print(dash)
print('\n\n')
print('{:<10}{:<50}{:>40}'.format('Site ID', 'Mailing Addresses', 'Mailing Company')+'\n')
for item in userCheck :
    print('{:<10}{:<50}{:>40}'.format(item['sId'], item['m1'], item['mC1']))
    print('{:<10}{:<50}{:>40}'.format('', item['m2'], item['mC2'])+'\n')

CSV retrieved.
Values sorted by siteid
Filled null sendnotices row values with zeros
Changed sendnotice rows to booleans
Structured intermediary target dataframe
Structured final dataframe
Got list of cities that supply their own envelopes
Added letter going to site: 1000033
Added letter going to site: 1000181
Added letter going to site: 1000193
Added letter going to site: 1000214
Added letter going to site: 1000310
Added letter going to site: 1000367
Added letter going to site: 1000820
Added letter going to site: 1000853
Added letter going to site: 1001090
Added letter going to site: 1001790
Added letter going to site: 1001792
Added letter going to site: 1001837
Added letter going to site: 1134356
Added letter going to site: 1136918
Added letter going to site: 1142417
Added letter going to site: 1147973
Added letter going to site: 1156060
Added letter going to site: 1158238
Added letter going to site: 1162840
Added letter going to site: 1163140
Added letter going to site: 1164424
Adde

Added letter going to site: 1298476
Added letter going to site: 1298502
Added letter going to site: 3016835
Added letter going to site: 3016836
Added letter going to site: 3023235
Added letter going to site: 3023261
Added letter going to site: 3023343
Added letter going to site: 3024241
Added letter going to site: 3024260
Added letter going to site: 3024399
Added letter going to site: 3024401
Added letter going to site: 3024407
Added letter going to site: 3024474
Added letter going to site: 3024484
Added letter going to site: 3024548
Added letter going to site: 3024648
Added letter going to site: 3024744
Added letter going to site: 3024789
Added letter going to site: 3024885
Added letter going to site: 3024907
Added letter going to site: 3025013
Added letter going to site: 3025059
Added letter going to site: 3025079
Added letter going to site: 3025674
Added letter going to site: 3025676
Added letter going to site: 3025907
Added letter going to site: 3026235
Added letter going to site: 

Added letter going to site: 3103021
Added letter going to site: 3103043
Added letter going to site: 3103128
Added letter going to site: 3103340
Added letter going to site: 3103448
Added letter going to site: 3103488
Added letter going to site: 3103492
Added letter going to site: 3103699
Added letter going to site: 3103760
Added letter going to site: 3104365
Added letter going to site: 3104387
Added letter going to site: 3104392
Added letter going to site: 3104440
Added letter going to site: 3104444
Added letter going to site: 3104459
Added letter going to site: 3104468
Added letter going to site: 3104473
Added letter going to site: 3104477
Added letter going to site: 3104526
Added letter going to site: 3104646
Added letter going to site: 3104670
Added letter going to site: 3104739
Added letter going to site: 3104749
Added letter going to site: 3104760
Added letter going to site: 3104831
Added letter going to site: 3104866
Added letter going to site: 3104960
Added letter going to site: 

Added letter going to site: 4003840
Added letter going to site: 4003863
Added letter going to site: 4003923
Added letter going to site: 4003984
Added letter going to site: 4004019
Added letter going to site: 4004024
Added letter going to site: 4004305
Added letter going to site: 4004310
Added letter going to site: 4004311
Added letter going to site: 4004315
Added letter going to site: 4004317
Added letter going to site: 4004521
Added letter going to site: 4004691
Added letter going to site: 4004692
Added letter going to site: 4004693
Added letter going to site: 4004695
Added letter going to site: 4004698
Added letter going to site: 4004699
Added letter going to site: 4004789
Added letter going to site: 4004932
Added letter going to site: 4022517
Added letter going to site: 4023171
Added letter going to site: 4023281
Added letter going to site: 4023490
Added letter going to site: 4023739
Added letter going to site: 4026157
Added letter going to site: 4030092
Added letter going to site: 

Added letter going to site: 4141201
Added letter going to site: 4141221
Added letter going to site: 4141291
Added letter going to site: 4141301
Added letter going to site: 4141320
Added letter going to site: 4141339
Added letter going to site: 4141477
Added letter going to site: 4141482
Added letter going to site: 4141521
Added letter going to site: 4141523
Added letter going to site: 4141589
Added letter going to site: 4141679
Added letter going to site: 4141701
Added letter going to site: 4141790
Added letter going to site: 4141915
Added letter going to site: 4141941
Added letter going to site: 4141946
Added letter going to site: 4142022
Added letter going to site: 4142035
Added letter going to site: 4142081
Added letter going to site: 4142316
Added letter going to site: 4142401
Added letter going to site: 4143416
Added letter going to site: 4143793
Added letter going to site: 4144192
Added letter going to site: 4144299
Added letter going to site: 4144335
Added letter going to site: 

...

Added letter going to mailing address: 1009 COLUMBINE CT
Added letter going to mailing address: 1040 VERMONT ST
Added letter going to mailing address: 1642 Bobwhite Drive
Added letter going to mailing address: 2337 N DIAMOND LN
Added letter going to mailing address: 1816 FOXFIRE DR
Added letter going to mailing address: 530 MCCALL RD STE 110
Added letter going to mailing address: 204 KNOTTED BEND CT
Added letter going to mailing address: 2101 MASSACHUSETTS ST
Added letter going to mailing address: 1100 MASSACHUSETTS ST
Added letter going to mailing address: PO BOX 708
Added letter going to mailing address: 213 KNOTTED BEND CT
Added letter going to mailing address: 4533 GROVE DR
Added letter going to mailing address: 2651 ARKANSAS ST
Added letter going to mailing address: 3805 STETSON DR
Added letter going to mailing address: 3014 HARPER ST
Added letter going to mailing address: 305 EDINBURGH RD
Added letter going to mailing address: 3511 TAM OSHANTER DR
Added letter going to mailing ad

Added letter going to mailing address: 409 MAIN ST
Added letter going to mailing address: PO BOX 128
Added letter going to mailing address: 6565 N BRADY ST
Added letter going to mailing address: 230 BERRY PKWY
Added letter going to mailing address: 209 N BROADWAY AVE
Added letter going to mailing address: 1024 W CRESCENT AVE
Added letter going to mailing address: 835 S PROSPECT AVE
Added letter going to mailing address: 2101 MONTCALM ST
Added letter going to mailing address: 3115 N POST RD
Added letter going to mailing address: 501 N COLLEGE AVE UNIT A
Added letter going to mailing address: 680 Meade Drive Suite 5
Added letter going to mailing address: 1203  WESCOTT DR
Added letter going to mailing address: 1 OAKBROOK TERRACE SUITE 600
Added letter going to mailing address: 1 PIERCE PLACE SUITE 450
Added letter going to mailing address: 101 W ELM ST STE 600
Added letter going to mailing address: 1200 GREENLEAF AVE
Added letter going to mailing address: 1201 TONNE RD
Added letter going 

Added letter going to mailing address: 11416 SANDYVILLE ROAD
Added letter going to mailing address: 11503 1ST STREET NE
Added letter going to mailing address: PO BOX 766
Added letter going to mailing address: PO BOX 116
Added letter going to mailing address: PO BOX 175
Added letter going to mailing address: 162 A ST RT 212 NE
Added letter going to mailing address: 162 B ST RT 212 NE
Added letter going to mailing address: 1686 MAPLE SQ NE
Added letter going to mailing address: 1835 DOVER-ZOAR RD.
Added letter going to mailing address: 358 EDGEBROOK NE
Added letter going to mailing address: 387 EDGEBROOK NE
Added letter going to mailing address: 395 MAIN STREET
Added letter going to mailing address: 6545 EAST SPARTA AVE.
Added letter going to mailing address: 5371 S.R. 183 N.E.
Added letter going to mailing address: 541 OLDE ORCHARD DRIVE, NE
Added letter going to mailing address: PO BOX 294
Added letter going to mailing address: 9166 S.R. 800 N.E.
Added letter going to mailing address: 

Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; Indianapolis
Setting envelope for Archon - Citizens Utilities; India







----------------------------------------------------------------------------------------------------
These addresses looked similar
----------------------------------------------------------------------------------------------------



Site ID   Mailing Addresses                                                          Mailing Company

1000057   150 E Roosevelt Rd                                                           Boston Market
          151 E Roosevelt Rd                                                          Boston Chicken

1000189   510 E 22nd Street                                            Northlake Fairfield Plaza LLC
          510 E 22nd St                                                            Dr. Dino's Dental

1000551   405 Eisenhower Ln S                                                            Venture One
          455 Eisenhower Lane S                                                     Current Occupant

1000695   2003 Meyers Road                     

# Add to python file

### Workspace

In [17]:
mask1 = df1['sendnotices'] == False
mask2 = df1['sendnotices.1'] == False
dfBothZero = df1[mask1 & mask2]
for i in dfBothZero.index :
    if (dfBothZero.at[i, 'siteid'] in dfFinal['sId'].values) :
        continue
    else :
        dfTarget.loc[0, 'utility'] = dfBothZero.loc[i, 'municipality']
        dfTarget.loc[0, 'sId'] = dfBothZero.loc[i, 'siteid']
        dfTarget.loc[0, ['sCompany', 'mCompany']] = dfBothZero.loc[i, 'company']
        dfTarget.loc[0, 'mContact'] = dfBothZero.loc[i, 'contact']
        dfTarget.loc[0, ['sAddress', 'mAddress']] = dfBothZero.loc[i, 'address']
        dfTarget.loc[0, ['sCity', 'mCity']] = dfBothZero.loc[i, 'city']
        dfTarget.loc[0, ['sState', 'mState']] = dfBothZero.loc[i, 'state']
        dfTarget.loc[0, ['sZip', 'mZip']] = dfBothZero.loc[i, 'zip']
        dfTemp = dfBothZero[dfBothZero['siteid'] == dfBothZero.at[i, 'siteid']]
        dfTempTrimmed = dfTemp.drop_duplicates('hazid')
        for i in dfTempTrimmed.index :
            ci = 16
            while ci < 100000 :
                if (np.isnan(dfTarget.at[0, dfTarget.columns[ci]])) :
                    dfTarget.at[0, dfTarget.columns[ci]] = dfTempTrimmed.at[i, 'hazid']
                    dfTarget.at[0, dfTarget.columns[ci + 1]] = dfTempTrimmed.at[i, 'testdue']
                    dfTarget.at[0, dfTarget.columns[ci + 2]] = dfTempTrimmed.at[i, 'devsize']
                    dfTarget.at[0, dfTarget.columns[ci + 3]] = dfTempTrimmed.at[i, 'model']
                    dfTarget.at[0, dfTarget.columns[ci + 4]] = dfTempTrimmed.at[i, 'serialnum']
                    dfTarget.at[0, dfTarget.columns[ci + 5]] = dfTempTrimmed.at[i, 'hazardcat']
                    dfTarget.at[0, dfTarget.columns[ci + 6]] = dfTempTrimmed.at[i, 'location']
                    dfTarget.at[0, dfTarget.columns[ci + 7]] = dfTempTrimmed.at[i, 'lasttestcompany']
                    dfTarget.at[0, dfTarget.columns[ci + 8]] = dfTempTrimmed.at[i, 'lasttestcompanyphone']
                    ci += 1000000
                else :
                    ci += 9
        dfTarget.at[0, 'utility'] = dfBothZero.at[i, 'municipality']
        dfFinal = dfFinal.append(dfTarget, ignore_index = True)
        print('Added letter going to site: ' + str(dfTarget.at[0, 'sId']))
        dfTarget.drop(dfTarget.index[0], inplace = True)

Added letter going to site: 3037034
Added letter going to site: 3050594
Added letter going to site: 3069132
Added letter going to site: 3070247
Added letter going to site: 3127205
Added letter going to site: 3143426
Added letter going to site: 3236312
Added letter going to site: 3260000
Added letter going to site: 3943808
Added letter going to site: 4064620
Added letter going to site: 4201245
Added letter going to site: 4675465
Added letter going to site: 4676404
Added letter going to site: 4707964
Added letter going to site: 4751650
Added letter going to site: 4751652


In [162]:
for i in dfOnlyDups.index :
    maskSID = dfOnlyDups['sId'] == dfOnlyDups.at[i, 'sId']
    dfTrimmed = dfOnlyDups[maskSID]
    for j in dfTrimmed.index :
        for x in dfTrimmed.index :
            if (j == x) :
                continue
            if (fuzz.ratio(dfTrimmed.at[j, 'mAddress'].lower(), dfTrimmed.at[x, 'mAddress'].lower()) > 60) :
                if (fuzz.ratio(dfTrimmed.at[j, 'mCity'].lower(), dfTrimmed.at[x, 'mCity'].lower()) < 90) :
                    continue
                elif not (dfTrimmed.at[j, 'sId'] in userCheckSIDs) :
                    userCheckSIDs.append(dfTrimmed.at[j, 'sId'])
                    userCheck.append({'sId':dfTrimmed.at[j, 'sId'], 'm1':dfTrimmed.at[j, 'mAddress'], 'm2':dfTrimmed.at[x, 'mAddress'], 'mC1':dfTrimmed.at[j, 'mCompany'], 'mC2':dfTrimmed.at[x, 'mCompany']})
                x += 1
            x += 1

In [129]:
userCheck

[{'sId': 1000057,
  'm1': '151 E Roosevelt Rd',
  'm2': '150 E Roosevelt Rd',
  'mC1': 'Boston Chicken',
  'mC2': 'Boston Market'},
 {'sId': 1000189,
  'm1': '510 E 22nd St',
  'm2': '510 E 22nd Street',
  'mC1': "Dr. Dino's Dental",
  'mC2': 'Northlake Fairfield Plaza LLC'},
 {'sId': 1000551,
  'm1': '405 Eisenhower Ln S',
  'm2': '455 Eisenhower Lane S',
  'mC1': 'Venture One',
  'mC2': 'Current Occupant'},
 {'sId': 1134345,
  'm1': '606 W. Burlington',
  'm2': '620 W. Burlington',
  'mC1': '1st National Bank of La Grange',
  'mC2': '1st National Bank of La Grange'},
 {'sId': 1147973,
  'm1': '1192 CLARENCE AVE, UNIT 1',
  'm2': '1192 CLARENCE AVE 1',
  'mC1': 'LARRY CROSBY',
  'mC2': 'LARRY CROSBY'},
 {'sId': 1250294,
  'm1': '1417 PARAMOUNT PARKWAY',
  'm2': '1407 PARAMOUNT PKWY',
  'mC1': '1405 PARAMOUNT CONDO ASSOCIATION',
  'mC2': 'BERTS AUTO SERVICE'},
 {'sId': 1259365,
  'm1': '77 N 1st St',
  'm2': '17 N 1st St',
  'mC1': 'Shodeen Group',
  'mC2': 'Shodeen'},
 {'sId': 3127042

In [163]:
dash = '-' * 100
print('\n\n\n\n\n\n' + dash)
print('These addresses look similar')
print(dash)
print('\n\n')
print('{:<10}{:<50}{:>40}'.format('Site ID', 'Mailing Addresses', 'Mailing Company')+'\n')
for item in userCheck :
    print('{:<10}{:<50}{:>40}'.format(item['sId'], item['m1'], item['mC1']))
    print('{:<10}{:<50}{:>40}'.format('', item['m2'], item['mC2'])+'\n')







----------------------------------------------------------------------------------------------------
These addresses look similar
----------------------------------------------------------------------------------------------------



Site ID   Mailing Addresses                                                          Mailing Company

1000057   151 E Roosevelt Rd                                                          Boston Chicken
          150 E Roosevelt Rd                                                           Boston Market

1000189   510 E 22nd St                                                            Dr. Dino's Dental
          510 E 22nd Street                                            Northlake Fairfield Plaza LLC

1000551   405 Eisenhower Ln S                                                            Venture One
          455 Eisenhower Lane S                                                     Current Occupant

1000695   2003 Meyers Road                       

In [63]:
for i in dfOnlyDups.index :
    maskSID = dfOnlyDups['sId'] == dfOnlyDups.at[i, 'sId']
    dfTrimmed = dfOnlyDups[maskSID]

In [91]:
userCheck

[1000057,
 1134345,
 1259365,
 3127362,
 3166364,
 3967578,
 3967677,
 3994278,
 1000189,
 1000551,
 1147973,
 1250294,
 3127042]

In [37]:
dfOnlyDups = dfTest[dfTest.duplicated(subset = 'sId', keep = False)]

In [54]:
for i in dfOnlyDups.index :
    maskSID = dfOnlyDups['sId'] == dfOnlyDups.at[i, 'sId']
    dfTempTrimmed = dfOnlyDups[maskSID]
#     for j in dfTempTrimmed.index :
#         print(j)

2477    4747531
4378    4747531
Name: sId, dtype: object


C:\Users\cameron\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [19]:
for i in dfTest['sId'].index :
    if not dfTest['sId'].is_unique :
        print(dfTest['sId'][i])

1000033
1000181
1000193
1000214
1000310
1000367
1000820
1000853
1001090
1001790
1001792
1001837
1134356
1134419
1134501
1136918
1137831
1142417
1147973
1156060
1158238
1162840
1163140
1164424
1165721
1169230
1175959
1201293
1201349
1201547
1201645
1201777
1201824
1201994
1202033
1202044
1202054
1220061
1236537
1245603
1245772
1245849
1245869
1246054
1246102
1246116
1246372
1246683
1247343
1247675
1249535
1250294
1250653
1250851
1251149
1251156
1251238
1251566
1252539
1253003
1253685
1256795
1257484
1258216
1258245
1258408
1258483
1258663
1258669
1258705
1258740
1259218
1259365
1259389
1259404
1259410
1259415
1259660
1259710
1259764
1260023
1260809
1260918
1260947
1260977
1261175
1261472
1261512
1261562
1261905
1262593
1265686
1265779
1265850
1265973
1266030
1266079
1266206
1266215
1266344
1266406
1266457
1266512
1266548
1266726
1266741
1266834
1267057
1269560
1270756
1272372
1273740
1273909
1274024
1274309
1274485
1274649
1274868
1274969
1275036
1275094
1275448
1275522
1275556
1275702


4188658
4188663
4188720
4188778
4188829
4188999
4189029
4189064
4189076
4189105
4189120
4189130
4189219
4189234
4189245
4189247
4189280
4189285
4189368
4189372
4189414
4189513
4189634
4189717
4189727
4189779
4189780
4189783
4189824
4189837
4189842
4189865
4189900
4189921
4189980
4189989
4190054
4190265
4190320
4190376
4190388
4190515
4190557
4190666
4190707
4190719
4190741
4190792
4190817
4190821
4190958
4190978
4191093
4191125
4191133
4191144
4191180
4191192
4191227
4191279
4191299
4191306
4191316
4191328
4191347
4191357
4191370
4191383
4191452
4191545
4191595
4191598
4191600
4191680
4191741
4191745
4191758
4191796
4191867
4191884
4191911
4192016
4192028
4192050
4192142
4192190
4192200
4192504
4192558
4192589
4192617
4192665
4192698
4192725
4192728
4192731
4192781
4192782
4192906
4192930
4192966
4192972
4193125
4193154
4193167
4193221
4193272
4193279
4193327
4193366
4193425
4193428
4193450
4193474
4193483
4193502
4193512
4193516
4193517
4193519
4193521
4193539
4193545
4193567
4193590


3086211
3086263
3086430
3086760
3087226
3088006
3088070
3088086
3088123
3088124
3088140
3088147
3088160
3088165
3088168
3088170
3088183
3088184
3088191
3088206
3088408
3088417
3088418
3088419
3088421
3088424
3088425
3088426
3088428
3088429
3088430
3088432
3088456
3092696
3093193
3093199
3093235
3093291
3093442
3093442
3093934
3094578
3097745
3098056
3098186
3098275
3098543
3098599
3101545
3104831
3107704
3126921
3126923
3126923
3126925
3126985
3126987
3126991
3127042
3127046
3127048
3127049
3127056
3127079
3127087
3127088
3127096
3127115
3127115
3127116
3127117
3127118
3127119
3127145
3127181
3127181
3127181
3127234
3127235
3127239
3127239
3127240
3127274
3127281
3127318
3127321
3127329
3127356
3127360
3127362
3127362
3127379
3127385
3127414
3127426
3127430
3127441
3127442
3127444
3127447
3127523
3127597
3127599
3127641
3127647
3127673
3127673
3127702
3127710
3127710
3127732
3127735
3127737
3127737
3127738
3127743
3127767
3127802
3127841
3127848
3127848
3127849
3127993
3128018
3128026


4658269
4659544
4675421
4675470
4675494
4675497
4675575
4675580
4675589
4675650
4675663
4675664
4675665
4675668
4675669
4675679
4675702
4675729
4675755
4675773
4675833
4675856
4675940
4676068
4676069
4676138
4676228
4676294
4676315
4676330
4676340
4676346
4676370
4676380
4676388
4676399
4676424
4676435
4676450
4676455
4676509
4682227
4682369
4683254
4688153
4693555
4693561
4693564
4694170
4694173
4694200
4694201
4694244
4694386
4694390
4694456
4694534
4694556
4694855
4694871
4694987
4695061
4695091
4695289
4695299
4695369
4695465
4695466
4695500
4695574
4695613
4695671
4695675
4695711
4695751
4695816
4696079
4696079
4696127
4697552
4697557
4697591
4697681
4697682
4697717
4697748
4701582
4702978
4703056
4703109
4703190
4703322
4703493
4703516
4703553
4703569
4703801
4703827
4704072
4704093
4704176
4704634
4705858
4706426
4707114
4707123
4714442
4716790
4730151
4732508
4733326
4733328
4733336
4733348
4733383
4733771
4737754
4747531
4747551
4747560
4747565
4747566
4747568
4747574
4747576


In [12]:
dfTest['sId'].value_counts()

3127181    3
3036104    3
1000057    3
1000181    3
1002100    3
          ..
4699396    1
4699395    1
3069186    1
4699393    1
3069547    1
Name: sId, Length: 4299, dtype: int64

In [122]:
%%time
for i in dfTest['mAddress'].index :
    j = i+1
    while j < len(dfTest['mAddress']) :
        if i == j :
            j += 1
            continue
        if (fuzz.ratio(dfTest.at[i, 'mAddress'], dfTest.at[j, 'mAddress']) > 95) :
            print(dfTest.at[i, 'mAddress'])
            print(dfTest.at[j, 'mAddress'])
        j += 1

10 W State St
101 W State St
54  TANGLEWOOD DR
154  TANGLEWOOD DR
73  TANGLEWOOD DR
173  TANGLEWOOD DR
83  TANGLEWOOD DR
183  TANGLEWOOD DR
1651 N MAIN ST
151 N MAIN ST
1825 ENTERPRISE PKWY
1825 ENTERPRISE PKWY
905 W ALGONQUIN RD
95 W ALGONQUIN RD
1205 S EAST AVE
105 S EAST AVE
1800 E MAIN ST
100 E MAIN ST
1600 E MAIN ST
100 E MAIN ST
3 S404 SADDLE RIDGE COURT
3 S408 SADDLE RIDGE COURT
3 S412 SADDLE RIDGE COURT
3 S415 SADDLE RIDGE COURT
3 S412 SADDLE RIDGE COURT
3 S413 SADDLE RIDGE COURT
3 S415 SADDLE RIDGE COURT
3 S413 SADDLE RIDGE COURT
27 W344 GALUSHA ROAD; METER A
27 W344 GALUSHA ROAD; METER B
3 S406 SADDLE RIDGE COURT;
3 S403 SADDLE RIDGE COURT;
2 S409 EMERALD GREEN DRIVE A
2 S409 EMERALD GREEN DRIVE F
3535 LINCOLN PARK DR - IRR 001
3535 LINCOLN PARK DR - IRR 004
1 PARK ROW ST
1 PARK ROW ST
16121 CHANCELLORS RIDGE WAY
16131 CHANCELLORS RIDGE WAY
20120 PRESCOTT PLACE DR
20142 PRESCOTT PLACE DR
20142 PRESCOTT PLACE DR
20143 PRESCOTT PLACE DR
20142 PRESCOTT PLACE DR
20164 PRESCOTT PL

#### Method below is about twice as fast

In [106]:
sample = dfTest.head(500).copy()

In [121]:
%%time
for i in sample['mAddress'].index :
    j = i+1
    while j < len(sample['mAddress']) :
        if i == j :
            j += 1
            continue
        if (fuzz.ratio(sample.at[i, 'mAddress'], sample.at[j, 'mAddress']) > 95) :
            print(sample.at[i, 'mAddress'])
            print(sample.at[j, 'mAddress'])
        j += 1

10 W State St
101 W State St
54  TANGLEWOOD DR
154  TANGLEWOOD DR
73  TANGLEWOOD DR
173  TANGLEWOOD DR
83  TANGLEWOOD DR
183  TANGLEWOOD DR
1825 ENTERPRISE PKWY
1825 ENTERPRISE PKWY
Wall time: 8.58 s


In [73]:
%%time
for i in sample['mAddress'].index :
    for j in sample['mAddress'].index :
        if i == j :
            continue
        if (fuzz.ratio(sample.at[i, 'mAddress'], sample.at[j, 'mAddress']) > 95) :
            print(sample.at[i, 'mAddress'])
            print(sample.at[j, 'mAddress'])

10 W State St
101 W State St
101 W State St
10 W State St
54  TANGLEWOOD DR
154  TANGLEWOOD DR
73  TANGLEWOOD DR
173  TANGLEWOOD DR
83  TANGLEWOOD DR
183  TANGLEWOOD DR
154  TANGLEWOOD DR
54  TANGLEWOOD DR
173  TANGLEWOOD DR
73  TANGLEWOOD DR
183  TANGLEWOOD DR
83  TANGLEWOOD DR
1825 ENTERPRISE PKWY
1825 ENTERPRISE PKWY
1825 ENTERPRISE PKWY
1825 ENTERPRISE PKWY
Wall time: 16 s


In [28]:
%%time
for i in dfTest['mAddress'].index :
    for j in dfTest['mAddress'].index :
        if i == j :
            continue
        if (fuzz.ratio(dfTest.at[i, 'mAddress'], dfTest.at[j, 'mAddress']) > 99) :
            print(dfTest.at[i, 'mAddress'])
            print(dfTest.at[j, 'mAddress'])

1825 ENTERPRISE PKWY
1825 ENTERPRISE PKWY
1825 ENTERPRISE PKWY
1825 ENTERPRISE PKWY
1 PARK ROW ST
1 PARK ROW ST
1 PARK ROW ST
1 PARK ROW ST
2501 E 86TH ST
2501 E 86TH ST
2501 E 86TH ST
2501 E 86TH ST
1323 N BALDWIN AVE
1323 N BALDWIN AVE
1323 N BALDWIN AVE
1323 N BALDWIN AVE
1700 E 38TH ST
1700 E 38TH ST
1700 E 38TH ST
1700 E 38TH ST
Wall time: 24min 18s
